# Baseline

**Metodologia**: Como baseline, iremos recomendar os livros mais bem avaliados.

## 0. Setup

In [1]:
%load_ext autotime

time: 190 µs (started: 2024-02-17 23:19:10 -03:00)


In [2]:
#---- Bibliotecas:

# Análise e manipulação de dados:

import pandas as pd
import numpy as np

# Visualização de dados:

import plotly.io as pio
import plotly.express as px

time: 484 ms (started: 2024-02-17 23:19:10 -03:00)


In [3]:
#---- Data path:

data_path = '../../03-data/'

#---- Template dos gráficos:

pio.templates.default = "plotly_white"

time: 148 ms (started: 2024-02-17 23:19:10 -03:00)


## 1. Extração dos dados

In [4]:
dados = pd.read_parquet(data_path + '02-processed/01-baseline/01-baseline-data.parquet')

dados.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,NaN,tyler,texas,usa
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,NaN,seattle,washington,usa
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,16.0,h,new south wales,australia
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,16.0,rijeka,n/a,croatia
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,16.0,rijeka,n/a,croatia


time: 1.13 s (started: 2024-02-17 23:20:50 -03:00)


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 11 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   User-ID              1149780 non-null  int64  
 1   ISBN                 1149780 non-null  object 
 2   Book-Rating          1149780 non-null  int64  
 3   Book-Title           1031136 non-null  object 
 4   Book-Author          1031134 non-null  object 
 5   Year-Of-Publication  1031136 non-null  object 
 6   Publisher            1031134 non-null  object 
 7   Age                  840288 non-null   float64
 8   city                 1149780 non-null  object 
 9   state                1149780 non-null  object 
 10  country              1149780 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 96.5+ MB
time: 383 ms (started: 2024-02-17 23:21:01 -03:00)


## 2. Baseline

## 2.1. Escolha dos top 5, 10 e 20 livros 

**Regras**: 

- Melhores avaliações (média)
- Ter sido avaliado por 10 ou mais usuários

In [8]:
top20_books = dados\
    .groupby('Book-Title', as_index = False)\
    .agg(
        media_rating = ('Book-Rating', 'mean'),
        frequencia = ('ISBN', 'count')
    )\
    .query('frequencia > 10')\
    .sort_values(by = 'media_rating', ascending = False)\
    .head(20)['Book-Title']\
    .to_list()

top20_books

['The Blue Day Book: A Lesson in Cheering Yourself Up',
 'The Baby Book: Everything You Need to Know About Your Baby from Birth to Age Two',
 'Les Fleurs Du Mal',
 'The Lorax',
 'Die unendliche Geschichte: Von A bis Z',
 'Marching Through Culpeper : A Novel of Culpeper, Virginia, Crossroads of the Civil War',
 'Chobits (Chobits)',
 'Free',
 'Harry Potter Und Der Feuerkelch',
 "There's Treasure Everywhere--A Calvin and Hobbes Collection",
 'Der Kleine Hobbit',
 'El Hobbit',
 'Warchild',
 "Beyond Civilization: Humanity's Next Great Adventure",
 "Ain't Nobody's Business If You Do : The Absurdity of Consensual Crimes in Our Free Country",
 'Jesus Freaks: DC Talk and The Voice of the Martyrs - Stories of Those Who Stood For Jesus, the Ultimate Jesus Freaks',
 'Ranma 1/2 (Ranma 1/2)',
 'Good Faeries Bad Faeries',
 'How the Grinch Stole Christmas!',
 'A Game of You (Sandman, Book 5)']

time: 881 ms (started: 2024-02-17 23:41:27 -03:00)


### 2.2. 

In [ ]:
dados\
    .groupby()